In [ ]:
"""General methyl data import using MSM package. General dictonary data, beta value methylation array df, survival metadata df"""

import sys
sys.path.insert(0, '/Users/zacksiegfried/Documents/methylspan')
import MSM

all_files = MSM.pullMethylMetaData('pancreas')

# not gaureteed to return same number of cases, (remove file_id from sample data if missing from meta data)
# missing values dropped anyways in next steps
sample_data = MSM.methylDataFormat(all_files, 30)
meta_data = MSM.metaDataFormat(sample_data, all_files)

In [ ]:
### PCA (NEEDS REWORK FOR DEAD + ALIVE CASES)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


def pcaModel(sample_df, meta_df):

    ### DEALING WITH MISSING VALUES
    # drop any cpg site that has one patient missing it
    sample_df = sample_df.dropna(axis=1, how='any')

    #scale predictor variables
    pca = PCA()
    X_reduced = pca.fit_transform(scale(sample_df))

    #define cross validation method
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

    regr = LinearRegression()
    mse = []

    # Calculate MSE with only the intercept
    score = -1 * model_selection.cross_val_score(regr, np.ones((len(X_reduced),1)), meta_df, cv=cv, scoring='neg_mean_squared_error').mean()    
    mse.append(score)

    # Calculate MSE using cross-validation, adding one component at a time 
    for i in np.arange(1, 30): # SETS PCA RANGE (ADDS TIME)
        score = -1*model_selection.cross_val_score(regr, X_reduced[:,:i], meta_df, cv=cv, scoring='neg_mean_squared_error').mean()
        mse.append(score)

    plt.plot(mse)
    plt.xlabel('Number of Principal Components')
    plt.ylabel('value')


pcaModel(sample_data, meta_data)

In [ ]:
### PCA scree plot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pcaTest(sample_df, components=25):

    ### DEALING WITH MISSING VALUES
    # drop any cpg site that has one patient missing it
    sample_df = sample_df.dropna(axis=1, how='any')

    #creates scaled version of DataFrame
    scaled_df = sample_df.copy()
    scaled_df = pd.DataFrame(StandardScaler().fit_transform(scaled_df), index=scaled_df.index, columns=scaled_df.columns)

    #scale predictor variables
    pca = PCA(n_components=components)
    pca.fit(scaled_df)

    output = pca.transform(scaled_df)

    PC_values = np.arange(pca.n_components_) + 1
    plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
    plt.title('Scree Plot')
    plt.xlabel('Principal Component')
    plt.ylabel('Variance Explained')
    plt.show()

    var = 0
    count = 0
    for i in pca.explained_variance_ratio_:
        var += i 
        count += 1
        if var > 0.5:
            break
    print("Number of PCs to achieve 0.5 explained variance: " + str(count))

    var = 0
    count = 0
    for i in pca.explained_variance_ratio_:
        var += i 
        count += 1
        if var > 0.8:
            break
    print("Number of PCs to achieve 0.8 explained variance: " + str(count))

pcaTest(sample_data, 20)